# Exploratory Data Analysis of a Hospital Network's Encounter-Level Dataset


Exploring the encounter-level dataset of a large hospital network. Performing descriptive and inferential statistics to understad the data, make findings, test some hypothesis and draw meaningful conclusions to help optimise capacity planning, care pathways, and readmission reduction using historical encounter-level data.

In [277]:
# importing all necessary packages
import warnings
warnings.filterwarnings("ignore") #disabling warnings in the notebook

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 1.0 Exploring the Encounter-Level Dataset
### 1.1 Importing and Preparing the Data

In [278]:
#importing the dataset
encounter_lvl_df = pd.read_csv("./data/synthea_healthcare_data.csv")
encounter_lvl_df.head()

,ID,PATIENT,START,STOP,MONTH,ENCOUNTERCLASS,PROVIDER,REASONCODE,REASONDESCRIPTION,LOS_DAYS,TOTAL_COST,AGE_AT_START,PRIMARY_DIAG_CODE,PRIMARY_DIAG_DESC,CONDITION_COUNT
0,748f8357-6cc7-551d-f31a-32fa2cf84126,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-02-17 05:07:38,2019-02-17 05:22:38,2019-02-01,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,NaN,NaN,0.010417,877.79,0.000000,410620009,Well child visit (procedure),NaN
1,5a4735ae-423f-6563-28ab-b3d11b49b2d4,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-03-24 05:07:38,2019-03-24 05:22:38,2019-03-01,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,NaN,NaN,0.010417,269.68,0.095825,410620009,Well child visit (procedure),NaN
2,0bee1ce6-3e2c-5506-f71c-a7ba8f64a3d3,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-05-26 05:07:38,2019-05-26 05:22:38,2019-05-01,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,NaN,NaN,0.010417,1292.75,0.268309,410620009,Well child visit (procedure),NaN
3,6e93bcf9-45a4-8528-0120-1c1eaa930faf,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-07-28 05:07:38,2019-07-28 05:22:38,2019-07-01,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,NaN,NaN,0.010417,1323.87,0.440794,410620009,Well child visit (procedure),NaN
4,8b6787c3-4316-a0cb-899d-4746525c319f,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-10-27 05:07:38,2019-10-27 05:22:38,2019-10-01,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,NaN,NaN,0.010417,831.76,0.689938,410620009,Well child visit (procedure),NaN


In [279]:
encounter_lvl_df.shape #showing the size of the dataset

(61459, 15)

In [280]:
encounter_lvl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61459 entries, 0 to 61458
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 61459 non-null  object 
 1   PATIENT            61459 non-null  object 
 2   START              61459 non-null  object 
 3   STOP               61459 non-null  object 
 4   MONTH              61459 non-null  object 
 5   ENCOUNTERCLASS     61459 non-null  object 
 6   PROVIDER           61459 non-null  object 
 7   REASONCODE         15957 non-null  float64
 8   REASONDESCRIPTION  15957 non-null  object 
 9   LOS_DAYS           61459 non-null  float64
 10  TOTAL_COST         61459 non-null  float64
 11  AGE_AT_START       61459 non-null  float64
 12  PRIMARY_DIAG_CODE  61459 non-null  int64  
 13  PRIMARY_DIAG_DESC  61459 non-null  object 
 14  CONDITION_COUNT    26904 non-null  float64
dtypes: float64(5), int64(1), object(9)
memory usage: 7.0+ MB


In [281]:
encounter_lvl_df.isna().sum() #show number of null/blank cells

ID                       0
PATIENT                  0
START                    0
STOP                     0
MONTH                    0
ENCOUNTERCLASS           0
PROVIDER                 0
REASONCODE           45502
REASONDESCRIPTION    45502
LOS_DAYS                 0
TOTAL_COST               0
AGE_AT_START             0
PRIMARY_DIAG_CODE        0
PRIMARY_DIAG_DESC        0
CONDITION_COUNT      34555
dtype: int64

`REASONCODE`, `REASONDESCRIPTION`, and `CONDITION_COUNT` contain very large number of blank rows which will be treated.

In [282]:
#Checking the format of the time series attributes (START and STOP)
type(encounter_lvl_df.START.iloc[0]), type(encounter_lvl_df.STOP.iloc[0])

(str, str)

As shown above, the time series attributes `START` and `STOP` as shown above are formated as strings and need to be reformated to date-time.

In [283]:
# Checking the data-type of the `REASONCODE` attribute
encounter_lvl_df.REASONCODE.dtype

dtype('float64')

As shown above, the values of the `REASONCODE` variable are wrongly imported as continous numerical veriables instead of discrete categorical numbers.

From the exploration steps above, the following were observed;
* The dataset contains 61,459 observations and 15 attributes.  
* The `ID` column represent the unique identifier of each patient.  
* The `START` and `STOP` dates show the entry and exit dates of each patient while the `MONTH` is the month of entry.   
* `LOS_DAYS` and `AGE_AT_START` are represented as a continuous variable in years, to accurately represent the months, days, and time difference. This will be retained in its original format for more precise analysis and modeling.

**The Identified Columns' Properties**

| Attribute         | Variable class    | Identified Issues
| ----------------- | ----------------- | -----------------
| ID                | Categorical       | None
| PATIENT           | Categorical       | None
| START             | Numerical         | Needs reformating to date-time
| STOP              | Numerical         | Needs reformating to date-time
| MONTH             | Categorical       | Needs reformating to the month's name
| ENCOUNTERCLASS    | Categorical       | None
| PROVIDER          | Categorical       | None
| REASONCODE        | Categorical       | Contains 45,502 blank cells and wrongly formated as continous numerical variable
| REASONDESCRIPTION | Categorical       | Contains 45,502 blank cells
| LOS_DAYS          | Numerical         | None
| TOTAL_COST        | Numerical         | None
| AGE_AT_START      | Numerical         | None
| PRIMARY_DIAG_CODE | Categorical       | None
| PRIMARY_DIAG_DESC | Categorical       | None
| CONDITION_COUNT   | Numerical         | Contains 34,555 blank cells

### 1.2 Cleaning the Data 
**1. Changing the data type of the `START` and `STOP` attributes from string to date-time**

In [284]:
import datetime as dt

def convert_2_date_time(date_time_str):
    return dt.datetime.strptime(date_time_str, "%Y-%m-%d %H:%M:%S")

In [285]:
encounter_lvl_df.START = encounter_lvl_df.START.apply(convert_2_date_time)
encounter_lvl_df.STOP = encounter_lvl_df.STOP.apply(convert_2_date_time)

encounter_lvl_df.START.iloc[0] #preview the result

Timestamp('2019-02-17 05:07:38')

**2. Changing the `MONTH` attribute to the corresponding month name**

In [286]:
def get_month_name(date_str):
    dateObj = dt.datetime.strptime(date_str, "%Y-%m-%d") #convert the date string to date data type
    return dateObj.strftime("%B") #get the month name

In [287]:
encounter_lvl_df.MONTH = encounter_lvl_df.MONTH.apply(get_month_name)
encounter_lvl_df.head()

,ID,PATIENT,START,STOP,MONTH,ENCOUNTERCLASS,PROVIDER,REASONCODE,REASONDESCRIPTION,LOS_DAYS,TOTAL_COST,AGE_AT_START,PRIMARY_DIAG_CODE,PRIMARY_DIAG_DESC,CONDITION_COUNT
0,748f8357-6cc7-551d-f31a-32fa2cf84126,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-02-17 05:07:38,2019-02-17 05:22:38,February,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,NaN,NaN,0.010417,877.79,0.000000,410620009,Well child visit (procedure),NaN
1,5a4735ae-423f-6563-28ab-b3d11b49b2d4,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-03-24 05:07:38,2019-03-24 05:22:38,March,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,NaN,NaN,0.010417,269.68,0.095825,410620009,Well child visit (procedure),NaN
2,0bee1ce6-3e2c-5506-f71c-a7ba8f64a3d3,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-05-26 05:07:38,2019-05-26 05:22:38,May,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,NaN,NaN,0.010417,1292.75,0.268309,410620009,Well child visit (procedure),NaN
3,6e93bcf9-45a4-8528-0120-1c1eaa930faf,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-07-28 05:07:38,2019-07-28 05:22:38,July,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,NaN,NaN,0.010417,1323.87,0.440794,410620009,Well child visit (procedure),NaN
4,8b6787c3-4316-a0cb-899d-4746525c319f,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-10-27 05:07:38,2019-10-27 05:22:38,October,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,NaN,NaN,0.010417,831.76,0.689938,410620009,Well child visit (procedure),NaN


**3. Treating the missing values in the `REASONCODE` and `REASONDESCRIPTION` attributes.**  

>* These are categorical attributes with 45,502 blank cells in each column. Removing these huge number of rows from the dataset may cause the data to be skewed and negatively affect the analysis of other important attributes and their relationships. Filling the blank cells with the median value may also skew the data due to the huge number of blanks. 
>* Since these two attributes are categorical attributes, a new category will be created in each to represent the unspecified variables. 
>* Whereever there is an observation with both the `REASONCODE` and `REASONDESCRIPTION` attributes missing. They will be filled with __"0"__ and __"Unspecified"__ respectively. 
>* If there are rows with only one of the two attributes missing, the rows will be dropped from the table.

In [288]:
encounter_lvl_df.loc[(encounter_lvl_df["REASONCODE"].isna() & encounter_lvl_df["REASONDESCRIPTION"].isna()), "REASONCODE"] = 0
encounter_lvl_df.loc[(encounter_lvl_df["REASONCODE"] == 0), "REASONDESCRIPTION"] = "Unspecified"
encounter_lvl_df.head()

,ID,PATIENT,START,STOP,MONTH,ENCOUNTERCLASS,PROVIDER,REASONCODE,REASONDESCRIPTION,LOS_DAYS,TOTAL_COST,AGE_AT_START,PRIMARY_DIAG_CODE,PRIMARY_DIAG_DESC,CONDITION_COUNT
0,748f8357-6cc7-551d-f31a-32fa2cf84126,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-02-17 05:07:38,2019-02-17 05:22:38,February,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,0.0,Unspecified,0.010417,877.79,0.000000,410620009,Well child visit (procedure),NaN
1,5a4735ae-423f-6563-28ab-b3d11b49b2d4,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-03-24 05:07:38,2019-03-24 05:22:38,March,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,0.0,Unspecified,0.010417,269.68,0.095825,410620009,Well child visit (procedure),NaN
2,0bee1ce6-3e2c-5506-f71c-a7ba8f64a3d3,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-05-26 05:07:38,2019-05-26 05:22:38,May,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,0.0,Unspecified,0.010417,1292.75,0.268309,410620009,Well child visit (procedure),NaN
3,6e93bcf9-45a4-8528-0120-1c1eaa930faf,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-07-28 05:07:38,2019-07-28 05:22:38,July,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,0.0,Unspecified,0.010417,1323.87,0.440794,410620009,Well child visit (procedure),NaN
4,8b6787c3-4316-a0cb-899d-4746525c319f,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-10-27 05:07:38,2019-10-27 05:22:38,October,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,0.0,Unspecified,0.010417,831.76,0.689938,410620009,Well child visit (procedure),NaN


In [289]:
print(f"Number of filled blanks in 'REASONCODE': {encounter_lvl_df[encounter_lvl_df["REASONCODE"] == 0]["REASONCODE"].count()}")
print(f"Number of filled blanks in 'REASONDESCRIPTION': {encounter_lvl_df[encounter_lvl_df["REASONDESCRIPTION"] == "Unspecified"]["REASONDESCRIPTION"].count()}")

Number of filled blanks in 'REASONCODE': 45502
Number of filled blanks in 'REASONDESCRIPTION': 45502


The total number of blank rows replaced in `REASONCODE`  and `REASONDESCRIPTION` is the same as the initially discorvered count in each column. That means there were no rows where only one of the two columns was blank. Hence, all the blank attributes have been filled with the new default categorical values for representing __unspecified__ attributes.

In [290]:
#Changing the REASONCODE's datatype from float (continous) to int (discrete) because its a categorical variable
encounter_lvl_df.REASONCODE = encounter_lvl_df.REASONCODE.apply(lambda x: int(x))
encounter_lvl_df.head(2)

,ID,PATIENT,START,STOP,MONTH,ENCOUNTERCLASS,PROVIDER,REASONCODE,REASONDESCRIPTION,LOS_DAYS,TOTAL_COST,AGE_AT_START,PRIMARY_DIAG_CODE,PRIMARY_DIAG_DESC,CONDITION_COUNT
0,748f8357-6cc7-551d-f31a-32fa2cf84126,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-02-17 05:07:38,2019-02-17 05:22:38,February,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,0,Unspecified,0.010417,877.79,0.000000,410620009,Well child visit (procedure),NaN
1,5a4735ae-423f-6563-28ab-b3d11b49b2d4,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-03-24 05:07:38,2019-03-24 05:22:38,March,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,0,Unspecified,0.010417,269.68,0.095825,410620009,Well child visit (procedure),NaN


**4. Treating the blank cells in `CONDITION_COUNT` attribute**

In [291]:
#checking the proportion of the missing values to filled values in the column
print(f"Number of blank cells in 'CONDITION_COUNT': {encounter_lvl_df.CONDITION_COUNT.isna().sum()}")
print(f"Scale of missing values in 'CONDITION_COUNT': {round((encounter_lvl_df.CONDITION_COUNT.isna().sum()/len(encounter_lvl_df))*100, 2)}%")

Number of blank cells in 'CONDITION_COUNT': 34555
Scale of missing values in 'CONDITION_COUNT': 56.22%


`CONDITION_COUNT` contains 34,555 blank cells which represent over 56% of the data in the attribute. Droping the missing rows may largely affect analysis of this dataset, hence, these values will be retained as missing and explicitly accounted for in analysis.

To enable analysis involving `CONDITION_COUNT`, an extra attribute called `CONDITION_COUNT_PRESENT` will be created to classify the present state of the condition count ('present' for non-blank, 'missing' for blank cells). This will enable easy filtering when needed.

In [292]:
#creating the extra column called CONDITION_COUNT_VALID for classifying the validity of each condition count cell
encounter_lvl_df["CONDITION_COUNT_PRESENT"] = np.where(encounter_lvl_df.CONDITION_COUNT.isna(), "missing", "present")
encounter_lvl_df.head(2)

,ID,PATIENT,START,STOP,MONTH,ENCOUNTERCLASS,PROVIDER,REASONCODE,REASONDESCRIPTION,LOS_DAYS,TOTAL_COST,AGE_AT_START,PRIMARY_DIAG_CODE,PRIMARY_DIAG_DESC,CONDITION_COUNT,CONDITION_COUNT_PRESENT
0,748f8357-6cc7-551d-f31a-32fa2cf84126,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-02-17 05:07:38,2019-02-17 05:22:38,February,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,0,Unspecified,0.010417,877.79,0.000000,410620009,Well child visit (procedure),NaN,missing
1,5a4735ae-423f-6563-28ab-b3d11b49b2d4,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-03-24 05:07:38,2019-03-24 05:22:38,March,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,0,Unspecified,0.010417,269.68,0.095825,410620009,Well child visit (procedure),NaN,missing


In [293]:
#Keep the blanks in `CONDITION_COUNT` as missing (NA)
encounter_lvl_df["CONDITION_COUNT"] = encounter_lvl_df["CONDITION_COUNT"].astype("Int64")
encounter_lvl_df.head(2)

,ID,PATIENT,START,STOP,MONTH,ENCOUNTERCLASS,PROVIDER,REASONCODE,REASONDESCRIPTION,LOS_DAYS,TOTAL_COST,AGE_AT_START,PRIMARY_DIAG_CODE,PRIMARY_DIAG_DESC,CONDITION_COUNT,CONDITION_COUNT_PRESENT
0,748f8357-6cc7-551d-f31a-32fa2cf84126,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-02-17 05:07:38,2019-02-17 05:22:38,February,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,0,Unspecified,0.010417,877.79,0.000000,410620009,Well child visit (procedure),<NA>,missing
1,5a4735ae-423f-6563-28ab-b3d11b49b2d4,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,2019-03-24 05:07:38,2019-03-24 05:22:38,March,wellness,82608ebb-037c-3cef-9d34-3736d69b29e8,0,Unspecified,0.010417,269.68,0.095825,410620009,Well child visit (procedure),<NA>,missing


**5. Checking for duplicates rows and removing them**

In [294]:
print(f"Number of duplicate rows: {encounter_lvl_df.duplicated().sum()}")

Number of duplicate rows: 0


There are no duplicate rows. Therefore, the dataframe has been completely cleaned and prepared for analysis.

**6. Performing quick exploration of the cleaned dataset.**

In [295]:
encounter_lvl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61459 entries, 0 to 61458
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   ID                       61459 non-null  object        
 1   PATIENT                  61459 non-null  object        
 2   START                    61459 non-null  datetime64[ns]
 3   STOP                     61459 non-null  datetime64[ns]
 4   MONTH                    61459 non-null  object        
 5   ENCOUNTERCLASS           61459 non-null  object        
 6   PROVIDER                 61459 non-null  object        
 7   REASONCODE               61459 non-null  int64         
 8   REASONDESCRIPTION        61459 non-null  object        
 9   LOS_DAYS                 61459 non-null  float64       
 10  TOTAL_COST               61459 non-null  float64       
 11  AGE_AT_START             61459 non-null  float64       
 12  PRIMARY_DIAG_CODE        61459 n

As shown in the meta-data above, the dataset now contains 15 attributes, all attributes are formatted properly to the appropriate data-type, and all blank rows are completely filled except '`CONDITION_COUNT`' because of the scale of the missing values and the statistical significance of not dropping or filling those rows to maintain the veracity and the distribution of the variables in the dataset.

In [296]:
encounter_lvl_df.describe().T

,count,mean,min,25%,50%,75%,max,std
START,61459,2005-03-13 05:03:48.042369792,1912-09-26 02:47:25,1997-07-29 19:18:21.500000,2012-10-29 13:24:37,2017-05-31 01:18:50.500000,2021-11-19 16:50:22,NaN
STOP,61459,2005-03-13 11:43:58.700304256,1912-09-26 03:02:25,1997-07-30 06:37:03.500000,2012-10-29 13:50:03,2017-05-31 01:33:50.500000,2021-11-19 17:05:22,NaN
REASONCODE,61459.0,1261982262255.819092,0.0,0.0,0.0,10509002.0,10939881000119104.0,108204083639761.90625
LOS_DAYS,61459.0,0.277901,0.010417,0.010417,0.010417,0.010417,5236.0,29.366192
TOTAL_COST,61459.0,4149.657952,0.0,129.16,786.33,1625.89,873646.19,10919.677889
AGE_AT_START,61459.0,42.991802,0.0,22.940452,42.031485,60.903491,109.94935,24.132002
PRIMARY_DIAG_CODE,61459.0,265270659.968776,1505002.0,162673000.0,185347001.0,390906007.0,702927004.0,157517077.726365
CONDITION_COUNT,26904.0,1.415923,1.0,1.0,1.0,2.0,13.0,0.753878


* The earliest admission start date is "1912-09-26 02:47:25" while the latest is "2021-11-19 16:50:22"
* The earliest admission stop date is "1912-09-26 03:02:25" while the latest is "2021-11-19 17:05:22"
* The highest Length-of-Stay days (LOS) is 5,236 days which is estimately 14 years
* The average length of Stay is 0.277901 (days) which is aproximately 6.67 hours

## 2.0 Performing Exploratory Data Analysis on the Dataset to Answer Specific Questions
### 2.1 Analysing and comparing the monthly fluctuations in admissions